In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/mlds


In [2]:
import data_auto_sys as aus
import tasker_link_prediction as t_lp
from splitter import splitter
from models import TwoStream_GCN
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_AS.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.005},
 'model_path': 'models/AS_TwoStream/',
 'classifier_hidden_size': 20,
 'ffn_fusion_size': 10,
 'ffn_hidden_size': 10,
 'gcn_fusion_size': 20,
 'spatial_hidden_size': 20,
 'spatial_input_dim': 128,
 'temporal_hidden_size': 20,
 'temporal_input_dim': 200,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.7,
 'dev_proportion': 0.1,
 'data_path': 'data/AS-733/',
 'prep_data_path': 'prep_data/AS733_neg/'}

In [4]:
data = aus.Autonomous_System_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= TwoStream_GCN(spatial_input_dim=config['spatial_input_dim'],
                 temporal_input_dim=config['temporal_input_dim'],
                 spatial_hidden_size=config['spatial_hidden_size'],
                 temporal_hidden_size=config['temporal_hidden_size'],
                 classifier_hidden_size=config['classifier_hidden_size'],
                 gcn_fusion_size=config['gcn_fusion_size'],
                 ffn_fusion_size=config['ffn_fusion_size'],
                 ffn_hiden_size=config['ffn_hidden_size'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])

Dataset splits sizes:  train 60 dev 10 test 20


In [5]:
trainer.resume_training(config['num_epochs'])

Epoch 43, Step 1, Loss: 0.0012227805564180017
Epoch 43, Step 2, Loss: 0.0012864758027717471
Epoch 43, Step 3, Loss: 0.0014129674527794123
Epoch 43, Step 4, Loss: 0.0016562173841521144
Epoch 43, Step 5, Loss: 0.001510271686129272
Epoch 43, Step 6, Loss: 0.001250241999514401
Epoch 43, Step 7, Loss: 0.0010615597711876035
Epoch 43, Step 8, Loss: 0.0012199723860248923
Epoch 43, Step 9, Loss: 0.0014406561385840178
Epoch 43, Step 10, Loss: 0.0016184308333322406
Epoch 43, Step 11, Loss: 0.0013877243036404252
Epoch 43, Step 12, Loss: 0.0012573410058394074
Epoch 43, Step 13, Loss: 0.0010303558083251119
Epoch 43, Step 14, Loss: 0.001065000076778233
Epoch 43, Step 15, Loss: 0.0012060647131875157
Epoch 43, Step 16, Loss: 0.0015957728028297424
Epoch 43, Step 17, Loss: 0.0016120313666760921
Epoch 43, Step 18, Loss: 0.0018505307380110025
Epoch 43, Step 19, Loss: 0.0013152349274605513
Epoch 43, Step 20, Loss: 0.0011734304716810584
Epoch 43, Step 21, Loss: 0.0012939516454935074
Epoch 43, Step 22, Loss: 

In [8]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.7424908046364151
Validation Metric MAPs: 0.815854149781898
Test Metric MRRs: 0.7388008140672709
Test Metric MAPs: 0.8125230379888071


(0.7388008140672709, 0.8125230379888071)